# Initial Roll Control Analysis  
*by William Harrington*  
*Portland State Aerospace Society*  

The purpose of this notebook is to take a detailed look at how the roll control system worked during Launch-12.  
  
We will start by extracting data from the csv files.


In [ ]:
# lets get everything set up

%matplotlib inline

from matplotlib import rcParams
rcParams.update({'font.size': 16})

# import numpy for numerical stuff
import numpy as np

# import matplotlib.pyplot for plotting stuff
import matplotlib.pyplot as plt

# import scipy.integrate.simps for integration
from scipy.integrate import simps

# import the sexy Savitzky-Golay filter method
from scipy.signal import savgol_filter

# get the ADIS data
columns = np.loadtxt('fc-data/ADIS.csv', delimiter=',', unpack=True)

# sort the data

# time
time = columns[1]

# gyro readings
gyro_x = columns[3]
gyro_y = columns[4]
gyro_z = columns[5]

# acceleromter readings
acc_x = columns[6]
acc_y = columns[7]
acc_z = columns[8]

# magnetometer readings
magn_x = columns[9]
magn_y = columns[10]
magn_z = columns[11]

### Accelerometer analysis

Now that we've got everything imported and data extracted from the csv files we can start analyzing it.  
  
Let's take a look at the accelerometer readings first as the velocity of the rocket and altitude of the rocket play an essential part in roll control authority.

In [ ]:
# umblicial disconnect event
t_0 = 117853569585227

# element wise subtraction
time = np.subtract(time, t_0)

# convert from ns to s
time = np.divide(time, 1e9)

# get rid of readings we don't care about
# launch begins around index 3650
# apogee occurs around index 28125
# I manually figured this out prior to writing this notebook..

time = time[3650:28125]
acc_x = acc_x[3650:28125]

# subtract gravity from x-axis accelerometer readings
acc_x = np.subtract(acc_x, 9.8)

# filter gyro readings
acc_x_savgol = savgol_filter(acc_x, 201, 2)
acc_x = acc_x_savgol

# array for holding first integration results
velocity = []

for i in range(len(acc_x)):
    # avoid integration error
    if(i > 0):
      # use simpsons rule on the samples to integrate
      velocity.append(simps(acc_x[:i], time[:i]))
        
# array for holding second integration results
position = []

for j in range(len(velocity)):
    # avoid integration error
    if(j > 0):
        # use simpsons rule on the samples to integrate
        position.append(simps(velocity[:j], time[:j]))

velocity = np.array(velocity)
position = np.array(position)

print 'Max velocity: %3.2f m/s' % velocity.max()
print 'Peak altitude: %3.2f m' % position.max()

In [ ]:
# now let's observe the curves
fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, sharex=True, figsize=(16,9))

#plt.figure(figsize=(16,9))
ax0.plot(time, acc_x)
ax0.set_title('Acceleration')
ax0.set_ylabel('Acceleration [$m/s^2$]')

ax1.plot(time[1:], velocity)
ax1.set_title('Velocity')
ax1.set_ylabel('Velocity [$m/s$]')

ax2.plot(time[2:], position)
ax2.set_title('Altitude')
ax2.set_ylabel('Altitude [$m$]')
ax2.set_xlabel('Time [$s$]')

plt.show()

These are the expected curves for velocity and altitude. This is a good sign as these played a crucial role in the simulation that was used for designing the roll control algorithm.  

### Gyro analysis

Now let's take a look at the gyro readings and see how the roll control algorithm performed.  

In [ ]:
# get rid of readings we don't care about
gyro_x = gyro_x[3650:28125]

# filter gyro readings
gyro_x_savgol = savgol_filter(gyro_x, 201, 2)

# filter derivative of gyro readings
angacc_savgol = savgol_filter(gyro_x, 201, 2, deriv=1, delta=1/819.2)

# use those sexy filtered readings
gyro_x = gyro_x_savgol
ang_acc = angacc_savgol

# integrate to get angular position
ang = []
for k in range(len(gyro_x)):
    if(k > 0):
        ang.append(simps(gyro_x[:k], time[:k]))

# now let's observe the curves
fig2, (ax3, ax4, ax5) = plt.subplots(nrows=3, sharex=True, figsize=(16,9))

ax3.plot(time[1:], ang)
ax3.set_title('Angular position')
ax3.set_ylabel('Angle [$deg$]')

ax4.plot(time, gyro_x)
ax4.set_title('Roll rate')
ax4.set_ylabel('Angular vel. [$deg/s$]')

ax5.plot(time, ang_acc)
ax5.set_title('Angular acceleration')
ax5.set_ylabel('Angular accel. [$deg/s^2$]')
ax5.set_ylim([-500, 500])

plt.show()

ang = np.array(ang)
gyro_x = np.array(gyro_x)
ang_acc = np.array(ang_acc)

print 'Max angular position: %3.2f degrees' % ang.max()
print 'Total revolutions: %3.2f' % (ang.max()/360.0)

print 'Max angular velocity: %3.2f deg/s' % gyro_x.max()
print 'Max angular acceleration: %3.2f deg/s^2' % ang_acc.max()

print 'Average angular velocity: %3.2f deg/s' % gyro_x.mean()
print 'Average angular acceleration: %3.2f deg/s^2' % ang_acc.mean()

  
These are some interesting results. The roll control algorithm was designed to prevent any revolutions at all.  
However, the rocket ended up spinning about 4 times according to this data.  
Since the average angular acceleration is close to 0, it looks like the algorithm was successful in keepting the roll rate relatively constant through certain periods of the flight.